Environment set (For Coolab)


In [4]:
!git clone https://github.com/Alisbasa/Ensayador.git
!pip install transformers==3.5.0
!pip install torch==1.4.0.
!pip install tokenizers==0.9.3

fatal: destination path 'Ensayador' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 22.4 MB/s 
     |████████████████████████████████| 2.9 MB 64.4 MB/s 
     |████████████████████████████████| 1.1 MB 54.0 MB/s 
     |████████████████████████████████| 880 kB 60.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=b8e5ef794702c45f726fdd27a69013d506832d22750f959f5dd3361aa2259e8e
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 753.4 MB 6.4 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Success

In [5]:
!pip install tokenizers==0.9.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import os
from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import BpeTrainer

class BPE_token(object):
    def __init__(self):
        self.tokenizer = Tokenizer(BPE())
        self.tokenizer.normalizer = Sequence([
            NFKC()
        ])
        self.tokenizer.pre_tokenizer = ByteLevel()
        self.tokenizer.decoder = ByteLevelDecoder()

    def bpe_train(self, paths):
        trainer = BpeTrainer(vocab_size=50000, show_progress=True, inital_alphabet=ByteLevel.alphabet(), special_tokens=[
            "<s>",
            "<pad>",
            "</s>",
            "<unk>",
            "<mask>"
        ])
        self.tokenizer.train(trainer, paths)

    def save_tokenizer(self, location, prefix=None):
        if not os.path.exists(location):
            os.makedirs(location)
        self.tokenizer.model.save(location, prefix)

In [7]:
from pathlib import Path
import os
# the folder 'text' contains all the files
paths = [str(x) for x in Path("./Ensayador/dataset").glob("**/*.txt")]
tokenizer = BPE_token()
# train the tokenizer model
tokenizer.bpe_train(paths)
# saving the tokenized data in our specified folder 
save_path = 'tokenized_data'
tokenizer.save_tokenizer(save_path)

#Model initialization

In [8]:
import tensorflow as tf
import torch.optim.lr_scheduler
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer
# loading tokenizer from the saved model path
tokenizer = GPT2Tokenizer.from_pretrained(save_path)
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})
# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id
)
# creating the model
model = TFGPT2LMHeadModel(config)

In [9]:
single_string = ''
for filename in paths:
  with open(filename, "r", encoding='utf-8') as f:
   x = f.read()
  single_string += x + tokenizer.eos_token
string_tokenized = tokenizer.encode(single_string)

In [10]:
examples = []
block_size = 100
BATCH_SIZE = 12
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
  examples.append(string_tokenized[i:i + block_size])
inputs, labels = [], []
for ex in examples:
  inputs.append(ex[:-1])
  labels.append(ex[1:])
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [11]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [ ]:
num_epoch = 5

checkpoint_filepath = '/content/check/chec2'
history = model.fit(dataset, epochs=num_epoch)


Epoch 1/5
13363/13363 [==============================] - 11464s 856ms/step - loss: 6.3209 - output_1_loss: 6.3209 - output_1_accuracy: 0.1512 - output_2_1_accuracy: 0.0024 - output_2_2_accuracy: 0.0023 - output_2_3_accuracy: 0.0028 - output_2_4_accuracy: 0.0024 - output_2_5_accuracy: 0.0026 - output_2_6_accuracy: 0.0025 - output_2_7_accuracy: 0.0025 - output_2_8_accuracy: 0.0024 - output_2_9_accuracy: 0.0026 - output_2_10_accuracy: 0.0023 - output_2_11_accuracy: 0.0027 - output_2_12_accuracy: 0.0024
Epoch 2/5
 1578/13363 [==>...........................] - ETA: 2:48:14 - loss: 5.7330 - output_1_loss: 5.7330 - output_1_accuracy: 0.1808 - output_2_1_accuracy: 0.0024 - output_2_2_accuracy: 0.0024 - output_2_3_accuracy: 0.0029 - output_2_4_accuracy: 0.0025 - output_2_5_accuracy: 0.0027 - output_2_6_accuracy: 0.0027 - output_2_7_accuracy: 0.0025 - output_2_8_accuracy: 0.0025 - output_2_9_accuracy: 0.0026 - output_2_10_accuracy: 0.0024 - output_2_11_accuracy: 0.0026 - output_2_12_accuracy: 0.

In [ ]:
text = "Necesito 3 cosas"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
beam_output = model.generate(
  input_ids,
  max_length = 240,
  num_beams = 5,
  temperature = 0.7,
  no_repeat_ngram_size=2,
  num_return_sequences=5
)

print(tokenizer.decode(beam_output[0]))

In [ ]:
from transformers import WEIGHTS_NAME, CONFIG_NAME
import os
output_dir = '/content/check'
# creating directory if it is not present
if not os.path.exists(output_dir):
  os.mkdir(output_dir)
model_to_save = model.module if hasattr(model, 'module') else model
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)
# save model and model configs
model.save_pretrained(output_dir)
model_to_save.config.to_json_file(output_config_file)
# save tokenizer
tokenizer.save_pretrained(output_dir)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)
model  =TFGPT2LMHeadModel.from_pretrained(output_dir)